In [23]:
// general includes
#include <iostream>   // std::cout|endl 
#include <vector>   // std::vector
#include <list>   // std::list
#include <string>
// using namespace std;
struct NoWidget {};
struct WidgetM {
    int m;
};

## Quick Recap: function templates

- *by-value* function parameters
  ```cpp
  template <typename AUTO> 
  void func(AUTO arg) {};
  ```
- lvalue reference function parameters
  ```cpp
  template <typename AUTO> 
  void func(AUTO& arg) {}; 
  ```
- forwarding-reference parameters (**not** rvalue references)
  ```cpp
  template <typename AUTO> 
  void func(AUTO&& arg) {};
  ``` 
- a template function definition alone **is not** a function definitions: 
- only when a template is used somewhere in a compilation unit an instantiation/definition happens.
- Does it work: compilation suceeds for any types, where body is "valid", eg
  
```c++
template <typename AUTO> AUTO funcc(AUTO arg){
  arg.blub(); //mirght not work
  return arg;
}

template <typename AUTO> void func(AUTO&& arg){ };//has to be movable!

```

```c++
void func1(Widget arg){}; //copy

void func2(const Widget& arg){}; //bind anything!, pass what you want

void func3(Widget&& arg){}; //must be something movable!!

```




## Goals for today
- Differentiate between class and function templates
- Understand the relation w.r.t. automatic deduction of template parameters via function/constructor arguments

# Class templates

In C++ not only functions can be templated but also classes [(cppref)](https://en.cppreference.com/w/cpp/language/class_template).
Let's look at a simple structure `Pair` which holds two members of the same type:

In [16]:
using T = double; //type alias

struct Pair {
  T first;
  T second;
  Pair operator+(const Pair &other) {
    return Pair{first + other.first, second + other.second};
  }
};

// usage
Pair p1{1.0, 2.0};
Pair p2{2.0, 1.0};
auto sum = p1 + p2;
auto x = sum.first;
//std::cout << x << std::endl;
std::cout << sizeof(x);


8

Assuming the implementation of `Pair` is identical for a supported range of types which are aimed to be available as a "pair", it is advantageous to make `Pair` a *class template* with a single template parameter `T`, which can look like this:

In [19]:
template <typename T = double> 
struct PairT {
  T first;
  T second;
  PairT operator+(const PairT &other) { 
    return PairT{first + other.first, second + other.second};
  }
  // PairT(const T &first, const T &second) :first(first), second(second) {} //constructor
};

The usage of this class template is slightly different from the non-templated version above:

In [18]:
{ // usage with `int`
  PairT<int> p1{1, 2}; 
  PairT<int> p2{2, 1};
  auto sum = p1 + p2;  
}

{ // usage with `double`
  PairT<double> p1{1, 2};
  PairT<double> p2{2, 1};
  auto sum = p1 + p2;
}

**Why do the template parameters above have to be explicitly provided above?**
- deduction rules don't allow deduction from list initialization [(cppref)](https://en.cppreference.com/w/cpp/language/list_initialization)


**Can this be avoided?**
- yes, by providing a user-defined constructor
  ```cpp
    PairT(const T &first, const T &second);
  ```
  This allows to avoid the explicit template arguments, as the deduction can be performed using the arguments of the constructor.

## Template argument determination
As for a *function template* also for a *class template* all arguments for the template parameters must be known to the compiler.
And again (as for function templates) there are three mechanism which lead to the determination of template arguments (precedence in this order):

- explicitly specifying the arguments on construction


In [ ]:
{
  PairT<int> p1{1, 2};               // instantiates PairT<int>
  PairT<double> p2{1, 2};            // instantiates PairT<double>
  PairT<std::string> p3{"1", "2"};   // instantiates PairT<std::string>
}

- deduction from constructor arguments

In [ ]:
{
  PairT p1{1, 2};                               // instantiates PairT<int>
  PairT p2{1.0, 2.0};                           // instantiates PairT<double>
  PairT p3{std::string("1"),std::string("2")};  // instantiates PairT<std::string> 
}

- default arguments for template parameters are defined

In [ ]:
  template <typename T = double> 
  struct Pair2 {
    T first;
    T second;
  };

  // usage
  Pair2<> p1{1,2};  // instantiates Pair<double>

**Why is `<>` required to use the default above?**
- without the `<>` constructor arguments would be used for deduction, which would result in `Pair<int>` 

**Can you imagine a situation where template deduction is not possible from the arguments passed to a constructor?**
- yes, default constructor (no arguments).
- yes, if there is no "link" between the types of the ctor parameters and the template parameters.

Let's look at some more situations of using the `Pair` class template where it might not be obvious which template argument is deduced:

In [25]:
{
  PairT p1{std::string("1"), std::string("2")};   // (1) instantiates Pair<???> 
  //PairT p2(std::string("12"), std::string("24")); // (2) instantiates Pair<???> 
  //PairT p3(1, "24");    // (3) instantiates Pair<???>
  //PairT p4(1.0, 1);     // (4) instantiates Pair<???>
  //PairT p5(1, 1.0);     // (4) instantiates Pair<???>
}

input_line_35:3:3: error: unknown type name 'PairT'; did you mean 'Pair'?
  PairT p1{std::string("1"), std::string("2")};   // (1) instantiates Pair<???> 
  ^~~~~
  Pair
input_line_24:3:8: note: 'Pair' declared here
struct Pair {
       ^
input_line_35:3:12: error: no viable conversion from 'std::string' (aka 'basic_string<char>') to '__cling_N517::T' (aka 'double')
  PairT p1{std::string("1"), std::string("2")};   // (1) instantiates Pair<???> 
           ^~~~~~~~~~~~~~~~
input_line_35:3:30: error: no viable conversion from 'std::string' (aka 'basic_string<char>') to '__cling_N517::T' (aka 'double')
  PairT p1{std::string("1"), std::string("2")};   // (1) instantiates Pair<???> 
                             ^~~~~~~~~~~~~~~~


Interpreter Error: 

**Can the templates above be instantiated? What are the deduced types?**
- see inline comments above
- background on strings/characters:
  - character literals are of type `char`
  - string literals are of type `char[N]`
  - a `std::string` can be constructed from `char[N]` or `char`
  ```cpp
  auto c = 'A';  // char
  auto cn = "AA"; // char[N]
  auto str1 = std::string('A');
  auto str2 = std::string("AA");
  ```

## Template argument deduction

The rules for argument deduction are nearly identical to the rules for function templates, just that the constructor arguments now take the place of the function arguments:

In [29]:
// function template
//template <typename AUTO> //
template <typename AUTO, typename AUTO2> //would work too
void func(AUTO arg1, AUTO2 arg2) {};

// class template
//template <typename AUTO> 
template <typename AUTO, typename AUTO2> //would work too

struct Widget{
    Widget(AUTO arg1, AUTO2 arg2) {};
    Widget(AUTO arg){};
};

The only further difference is that *forwarding references* are not available through a *class template* alone:

In [ ]:
// function template
template<typename T>
void func2(T &&arg) {}; // T&& is a forwarding reference

// class template
template<typename T>
struct Widget2{
    T member;
    Widget2(T &&arg) {};   // T&& is a rvalue reference  . NOT a fwd reference!
    void func(T &&arg) {}; // T&& is a rvalue reference  
};

**Why does this difference make sense?**
- a class template parameter is not tied to a single function but the whole class (including all member functions) and is determined at construction.
- after construction, all class template parameter (and therefore the arguments of all other non-template member functions) is fixed.

Let's briefly look at two scenarios for a class template with a single template parameter and a constructor with a single dependent argument.

In [32]:
template <typename AUTO> 
struct Widget3 {
   Widget3(const Widget3& arg) {}; // cpoy ctot
   Widget3(const AUTO& arg) {}; //NOT cpoy ctot

};

template <typename AUTO> 
struct Widget4 {
   Widget4(Widget4&& arg){}; //this is move ctor
   Widget4(AUTO&& arg) {};
};

**Are the above constructors copy and move constructors?**
- no, the template parameter is distinct from the class type.
- these are user-defined constructors.

## Multiple template parameters

A class template (or function template) can be defined using more than one template parameter. 
Let's modify the `Pair` class template from above to now have separate template parameters for each member:

In [2]:
template <typename FIRST, typename SECOND>
struct PairTT {
  FIRST first;
  SECOND second;
  PairTT(const FIRST &first, const SECOND &second)
      : first(first), second(second) {}
  auto operator+(const PairTT &other) { 
    return PairTT{first + other.first, second + other.second};
  }
};

// usage 
PairTT p1(1, 2.0);  // (1) instantiates Pair<int, double> 
//PairTT p2(2.0, 1);  // (2) instantiates Pair<double, int> 
//auto sum = p1 + p2; // (3) still works ? NO, no conversion

input_line_8:12:1: error: use of class template 'PairTT' requires template arguments
PairTT p1(1, 2.0);  // (1) instantiates Pair<int, double> 
^
input_line_8:2:8: note: template is declared here
struct PairTT {
       ^


Interpreter Error: 

**Did anything break by introducing the second template parameter?**
- In principle, due to availability of implicit conversions between `double` and `int` we maybe expect everything works.
- ... but due to the impl. of the `operator+` only identical (or implicitly convertible) types can be `added`
- Option 1: free function to add `Pair`s with different template types
- Option 2a: overload version of `operator+` member
- Option 2b: transform `operator+` to a member function template

## User-defined deduction guides
Often multiple user-defined constructors are present and not necessarily all (or not even one) of them allows a deduction of the template arguments:

In [ ]:
template <typename AUTO> 
struct Widget5 {
  Widget5(AUTO arg) {};              
  Widget5(double arg1, int arg2) {}; 
};

// usage
Widget5 w1(1.0);      // (1) works ?
Widget5 w2(1.0, 2);   // (2) works ?

**Does automatic type deduction for the class template parameter succeed in the snippet above? If not, why not?**
- type for `w2` cannot be deduced as there is no relation between the template type and the ctor args 

Let's now look at an example where the constructor is additionally templated to allow construction from a range of types. Further, we know all types we are interested in themselves define a nested type `value_type` which we would like to set/deduce as template parameter for `Widget6`:

In [ ]:
template <typename T> struct Widget6 {
  T m;
  Widget6(const Widget6 &o) {} // copy ctor
  template <typename OTHER> Widget6(const OTHER &o) {} // template ctor
};

// deduction guides
Widget6(const double &o)->Widget6<double>;
Widget6(const std::vector<double> &o)->Widget6<double>;
template <typename OTHER>
Widget6(const OTHER &o)->Widget6<typename OTHER::value_type>;

{ // usage
  Widget6 w1(1.0);
  Widget6 w2(std::vector<double>(1, 1.0));
  Widget6 w3(std::list<double>(1, 1.0));
}

## Types of template parameters
Up to now we only considered *type template parameters*: parameters which represent a type.
There are two further categories:
- non-type template parameters (i.e., values)
- template template parameters (i.e., a template type where the parameters are not settled)

### Non-type template parameters
The most common example for a non-type template parameters are integers or boolean values:

In [ ]:
template<typename T, int N>
struct Array{
    T data[N]; 
};

// usage
Array<double,10> w10; 
Array<double,100> w100;
using Vec4 = Array<double,4>;

**Does the standard library provide a class similar to `Array` above?**
- yes, `std::array` [(cppref)](https://en.cppreference.com/w/cpp/container/array)

### Template template parameters
An example for a template template parameter is in the following snippet:

In [3]:
template < template <typename ...> class CONTAINER = std::vector> //type not fixed inside
struct Widget8{ 
  CONTAINER<float> data1;  // using different value type but same container-template
  CONTAINER<double> data2; // using different value type but same container-template
};

// usage:
// Widget8<std::vector> w8;

This allows to use an actual template type (e.g., `std::vector` and not `std::vector<double>`) as template parameter.

**In the snippet above, is it actually required to use a template template parameter?**
- yes, if we want to provide a templated type (e.g., a container) directly as a parameter.
- alternatives?
  - pass two template type arguments instead

**What is the reason to use `...` in the above snippet above?**
- `...` solves the problem to account for the full set of template parameter (e.g., `std::allocator<T>` in the example above).

# Summary
- class vs funczion templates
  - template type fixed at construction
  - '&&' is not forwarding ref for a non-template member function
- auto deduction using ctor args
- auto dec. only possible with a relation from ctor args to template params